In [10]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as draw

circuit = Circuit(4, 4).H(0).CX(0, 1).CX(1, 2).CX(2, 3)
circuit.measure_all()

circuit.name = "my-first-circuit"

draw(circuit)

# ADD YOUR OWN PROJECT NAME HERE
project = qnx.projects.get_or_create(name="sQuid Game")
qnx.context.set_active_project(project)



import datetime

jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")




h1_config = qnx.QuantinuumConfig(device_name="H1-Emulator")

aer_config = qnx.AerStateConfig()


ref = qnx.circuits.upload(circuit=circuit, name=f"GHZ-Circuit-{jobname_suffix}")


draw(ref.download_circuit())



ref_compile_job = qnx.start_compile_job(
    circuits=[ref],
    backend_config=aer_config,
    optimisation_level=2,
    name=f"compilation-job-{jobname_suffix}"
)


qnx.jobs.wait_for(ref_compile_job)
ref_compiled_circuit = qnx.jobs.results(ref_compile_job)[0].get_output()



compiled_circuit = ref_compiled_circuit.download_circuit()

draw(compiled_circuit)



ref_execute_job = qnx.start_execute_job(
    circuits=[ref_compiled_circuit],
    n_shots=[100],
    backend_config=aer_config,
    name=f"execution-job-{jobname_suffix}"
)




qnx.jobs.status(ref_execute_job)
qnx.jobs.wait_for(ref_execute_job)
ref_result = qnx.jobs.results(ref_execute_job)[0]
backend_result = ref_result.download_result()
print(backend_result.get_counts())




InvalidResultType: BackendResult cannot produce result of type shots/counts.